In [180]:
import numpy as np
import math
from collections import namedtuple, defaultdict
from scipy.stats import multivariate_normal
import mdptoolbox.mdp as mdp
import copy
%matplotlib inline


# Discretize the space

In [199]:
MDPConstants = namedtuple('MDPConstants', ['δ_t','n_1', 'n_2', 'n_u', 'θ_max', 'v_max', 'u_max'])
#θ is for X1, angular velocity is for X2, control is for u
PendulumConstants = namedtuple('PendulumConstants', ['a', 'b', 'σ', 'k', 'r', 'γ'])
#the constants that describe the pendulum
DiscreteSpace = namedtuple('DiscreteSpace', ['X1_matrix', 'X2_matrix', 'U', 'Flat_board'])
#The space describe the state space and control space

myMDP = MDPConstants(δ_t=0.5, n_1=20, θ_max= math.pi, v_max=8, u_max=4, n_2=10, n_u=20)
myPendulum = PendulumConstants(a=1, b=0, σ=1, k=1, r=1, γ=0.3)

X1 = np.linspace(-myMDP.θ_max, myMDP.θ_max, myMDP.n_1) # one D array for X1
X2 = np.linspace(-myMDP.v_max, myMDP.v_max, myMDP.n_2) # one D array for X2

X1_matrix, X2_matrix = np.meshgrid(X1, X2) #create the state space for X meshgrid format
U = np.linspace(-myMDP.u_max, myMDP.u_max, myMDP.n_u)[:, None] # one D array for control U
# where row number is for X2, column number is for X1
Flat_board = [i for i in zip(X1_matrix.flat, X2_matrix.flat)]
mySpace = DiscreteSpace(X1_matrix = X1_matrix, X2_matrix = X2_matrix, U = U, Flat_board = Flat_board)


# Gaussian Distribution

In [200]:
#generating the probability transition
Dmn_row = len(mySpace.Flat_board)
Dmn_U = mySpace.U.shape[0]
myX1matrix = np.copy(mySpace.X1_matrix)
myX2matrix = np.copy(mySpace.X2_matrix)
cov = myPendulum.σ * myMDP.δ_t

Pr_transition = np.zeros((Dmn_U, Dmn_row, Dmn_row))
stage_cost_table = np.zeros((Dmn_U, Dmn_row))

#define stage cost function 
stage_cost = lambda x1, u: (1 - np.exp(myPendulum.k * (np.cos(x1)-1)) + \
                            myPendulum.r*u**2/2) * myMDP.δ_t
#prepare for f(X,U) fucntion, with single u input
f1 = myX2matrix
f2 = myPendulum.a * np.sin(myX1matrix) - myPendulum.b * myX2matrix
myX1 = np.array(mySpace.Flat_board)[:,0]

for u_i in range(Dmn_U):
    #implement state updates, X + f(X, U)*δ_t
    u = mySpace.U[u_i, 0]
    stage_cost_table[u_i, :] = stage_cost(myX1, u)
    
    f2_u = f2 + u
    X1means = mySpace.X1_matrix + f1 * myMDP.δ_t # angle should be in the interval [-π, π]
    X1means = (X1means + math.pi) % (2*math.pi) - math.pi

    X2means = mySpace.X2_matrix + f2_u * myMDP.δ_t 
    #anglur velocity should be in interval[-vmax, vmax]
    X2means[X2means<-myMDP.v_max] = -myMDP.v_max
    X2means[X2means>myMDP.v_max] = myMDP.v_max
    
    #following is my shifted state
    myXmeans = [x for x in zip(X1means.flat, X2means.flat)]
    
    for xmean_j, xmean in enumerate(myXmeans):
        #find samples under each xmean of Xmeans, with gaussian distribution
        pr_shiftpts = multivariate_normal.pdf\
        (mySpace.Flat_board, mean=xmean, cov=np.array([[1,0],[0,1]]))
        
        threshold =  multivariate_normal.pdf\
        (xmean, mean=xmean, cov=np.array([[1,0],[0,1]])) * 0.6
        
        mask = np.array(pr_shiftpts) > threshold
        #myXsamples = np.array(mySpace.Flat_board)[mask]
        
        pr_myXsamples = np.array(pr_shiftpts)[mask]
        #normalize these samples with corresponding probability so that sum of them is 1
        normalized_pr_myXsamples = pr_myXsamples / pr_myXsamples.sum()
        Pr_transition[u_i, xmean_j, :][mask] = normalized_pr_myXsamples
    #print(xmean_j)
    
    


199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199


# Value iteration and Policy Iteration

In [194]:
class optimal ():
    def __init__(self, transition, Cost, Dmn_U, Dmn_row):
        self.P = transition
        self.C = Cost
        self.A = Dmn_U
        self.S = Dmn_row
        
    def _clean(self):
        self.value = np.zeros((self.S, 1))
        self.J = np.zeros((self.S, 1))
        self.policy = None
        self.discount = 0.9
        self.epsilon = 0.001
        self.i = 0
    
    def _bellmanOperator(self, V):
        Q = np.zeros((self.A, self.S)) #A, S
        for aa in range(self.A):
            Q[aa] = self.C[aa].squeeze() + (self.discount * (self.P[aa]).dot(V)).squeeze()
            #1* 10000 = 1* 10000 + 10000* 10000 dot 10000 * 1 
        policy = Q.argmin(axis=0) #1*10000 in 1D in index, each column compare its rows
        value = Q.min(axis=0)[:, None] #1*10000 in 1D
        return policy, value
        
    def ValueIteration(self):
        self._clean()
        delta = None
        while True:
            self.i = self.i + 1
            V = self.value.copy()
            self.policy, self.value = self._bellmanOperator(V)
            delta = abs(self.value - V)
            if delta.all() < (1-self.discount)/self.discount:
                print(self.i)
                break
                
    def PolicyIteration(self):
        #initialize
        self._clean()
        delta = None
        policy_idx = np.random.choice(self.A, self.S)
        value_idx = np.arange(self.S)
        
        while True:
            self.i = self.i + 1
            J = self.J.copy()
            #Policy Evaluation
            mycost = self.C[policy_idx, value_idx] #1 * 10000
            myP = self.P[policy_idx, value_idx, :]
            J = (mycost.squeeze() + (self.discount * myP.dot(J)).squeeze())[:,None]
            #Policy Improvement
            self.policy, self.J = self._bellmanOperator(J)
            policy_idx = self.policy.copy()
            
            delta = abs(self.J - J)
            if delta.all() < (1-self.discount)/self.discount:
                print(self.i)
                break
            #after break, we have best policy for each states, and value for each states
                   
myV = optimal(Pr_transition, stage_cost_table, Dmn_U, Dmn_row)  
myV.ValueIteration()
#print(myV.value.squeeze())
#print(myV.policy.squeeze())


myP = optimal(Pr_transition, stage_cost_table, Dmn_U, Dmn_row)  
myP.PolicyIteration()
#print(myP.J.squeeze())
#print(myP.policy.squeeze())

 

315
[ 2.86146733  2.83197605  2.80619246  2.78821755  2.77042676  2.75044482
  2.71926111  2.67496403  2.62765415  2.59891109  2.60924721  2.66314527
  2.7412942   2.77755995  2.84188616  2.87791769  2.89549389  2.89523087
  2.88728403  2.86146733  2.80956538  2.82205224  2.8494855   2.87632239
  2.88575503  2.86173212  2.80015725  2.71152788  2.62514381  2.56744006
  2.57068221  2.63014068  2.71641113  2.80394287  2.86100609  2.87280468
  2.87219737  2.84413434  2.81866679  2.80956538  2.8623733   2.88752084
  2.9022985   2.89502296  2.87787566  2.84240439  2.77809075  2.74377854
  2.66484634  2.61046864  2.59919068  2.62741104  2.67346692  2.71768805
  2.74908825  2.76968113  2.78485242  2.80687114  2.83299726  2.8623733
  2.8532959   2.88715498  2.90065765  2.91512699  2.8979019   2.9104504
  2.83886198  2.76133455  2.67186547  2.58299985  2.54164928  2.5465763
  2.56014614  2.59303512  2.631598    2.67402946  2.72079875  2.76133528
  2.811434    2.8532959   2.96000304  3.0043831   

# stablize the Pendulum

In [209]:
######how to find uu_idx based on given x1 and given x2
test_X1 = np.linspace(-myMDP.θ_max, myMDP.θ_max, myMDP.n_1) # one D array for X1
test_X2 = np.linspace(-myMDP.v_max, myMDP.v_max, myMDP.n_2) # one D array for X2
len_X1 = len(test_X1) #width
print(len_X1)
len_X2 = len(test_X2) #height
print(len_X2)
policy_table = myP.policy.squeeze()


given_x1 = 0
given_x2 = 0
mytrajectory = [(given_x1, given_x2)]
count = 0
while count < 20:
    count = count + 1
    
    X1_idx = np.argmin(np.abs(test_X1 - given_x1))
    X2_idx = np.argmin(np.abs(test_X2 - given_x2))

    uu_idx = len_X1 * X2_idx + X1_idx #  width * point_height + point_x1 index

    uu = policy_table[uu_idx]
    dw = np.random.multivariate_normal(np.array([0,0]), \
                                       np.array([[myMDP.δ_t, 0],[0, myMDP.δ_t]]))
    fx1 = given_x2
    fx2 = myPendulum.a * math.sin(given_x1) - myPendulum.b * given_x2
    fx2_u = fx2 + uu
    
    given_x1 = given_x1 + fx1* myMDP.δ_t  + dw[0]
    given_x2 = given_x2 + fx2_u* myMDP.δ_t  + dw[1]
    given_x1 = (given_x1 + math.pi) % (2*math.pi) - math.pi
    
    if given_x2 < -myMDP.v_max:
        given_x2 = -myMDP.v_max
    elif given_x2 > myMDP.v_max:
        given_x2 = myMDP.v_max
 
    mytrajectory.append((given_x1, given_x2))
    

mytrajectory


20
10


[(0, 0),
 (-0.0048759630400669884, 4.1449652377490551),
 (2.0987153388495177, 8),
 (-0.16637693275387022, 8),
 (-1.5414598164559186, 8),
 (1.0614331480254338, 8),
 (-0.88919353151810832, 8),
 (-2.7160562538763191, 8),
 (1.2908142305328933, 8),
 (-1.4713059568296494, 8),
 (2.548548350317164, 8),
 (1.9426473720464923, 8),
 (-1.0623510987346734, 8),
 (-2.5977357744373126, 8),
 (2.0363825160254923, 8),
 (-0.9801246173182232, 8),
 (2.9602459470102094, 8),
 (0.18509928942028253, 8),
 (-2.9265987888838758, 8),
 (1.8806812212981168, 8),
 (-0.00088143013083730182, 8)]

# Test

In [210]:
print(Pr_transition.shape)
print(stage_cost_table.T.shape)
mybest = mdp.PolicyIteration(Pr_transition, -stage_cost_table.T, 0.9)


print(mybest.V)
print(mybest.policy)
mybest.iter



(20, 200, 200)
(200, 20)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
[10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 

0

In [188]:
dw = np.random.multivariate_normal(np.array([0,0]), \
                                   np.array([[myMDP.δ_t, 0],[0, myMDP.δ_t]]))
dw[0]

0.65320240682647912